In [ ]:
!pip install ipython_memory_usage pytz

In [ ]:
!wget https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/wykop--sample.txt
        

In [ ]:
!wget https://s3.eu-central-1.amazonaws.com/jkan.pl/data-science/uek/wykop.txt

In [ ]:
import ipython_memory_usage 
%ipython_memory_usage_start 

In [ ]:
from datetime import datetime

date_obj = datetime.fromisoformat('2019-11-16T09:03:01+01:00')
assert str(date_obj.date()) == '2019-11-16'
date_obj

In [ ]:
from collections import namedtuple
Wykop = namedtuple('Wykop', ['date', 'title', 'tags'])
def parse_wykop_row(row):
    try:
        return Wykop(datetime.fromisoformat(row[0]), row[2], row[1].split('||'))
    except ValueError:
        return None

In [ ]:
import functools
import pytz
import itertools
from collections import Counter
from pprint import pprint as pp

with open('wykop--sample.txt', 'r') as f:
    wykops = (row.split("\t") for row in f.readlines())
    wykop_objects = (parse_wykop_row(row) for row in wykops)
    wykop_objects = (wykop for wykop in wykop_objects if wykop is not None)
    just_november = (wykop for wykop in wykop_objects if (wykop.date >= datetime(2019, 11, 1, tzinfo=pytz.timezone('UTC'))))
    just_november = (wykop for wykop in wykop_objects if (wykop.date <= datetime(2019, 11, 30, tzinfo=pytz.timezone('UTC'))))
    topics = map(lambda x: x.tags, just_november)
    all_together = itertools.chain.from_iterable(topics)

    cnt = Counter()
    for topic in all_together:
        cnt[topic] += 1

In [ ]:
sorted(cnt.items(), key=lambda pair: pair[1], reverse=True)
cnt.most_common(10)

In [ ]:
%ipython_memory_usage_stop

In [ ]:
import functools
import pytz
import itertools
from collections import Counter

with open('wykop.txt', 'r') as f:
    wykops = [row.split("\t") for row in f.readlines()]
    wykop_objects = [parse_wykop_row(row) for row in wykops]
    wykop_objects = [wykop for wykop in wykop_objects if wykop is not None]
    topics = list(map(lambda x: x.tags, wykop_objects))
    all_together = list(itertools.chain.from_iterable(topics))

    cnt = Counter()
    for topic in all_together:
        cnt[topic] += 1

In [ ]:
cnt.most_common()

### LOGS

In [ ]:
!wget https://uek-krakow.s3-eu-west-1.amazonaws.com/data-science/logs/cdn-access.log

In [ ]:
with open('cdn-access.log', 'r') as f:
    lines = (line for line in f)
    print(next(lines))
    

In [ ]:
example_line = '''sg-cdn.uek.krakow.pl 10.20.2.31 - - [08/Oct/2020:08:35:58 +0200] "GET //xmlrpc.php?rsd HTTP/1.1" 404 27 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36" "0.000"'''

In [ ]:
extract_code(example_line)

In [ ]:
import re
lineformat = re.compile(r"""(?P<ipaddress>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(?P<dateandtime>\d{2}\/[a-z]{3}\/\d{4}:\d{2}:\d{2}:\d{2} (\+|\-)\d{4})\] ((\"(GET|POST) )(?P<url>.+)(http\/1\.1")) (?P<statuscode>\d{3}) (?P<bytessent>\d+) (["](?P<refferer>(\-)|(.+))["]) (["](?P<useragent>.+)["])""", re.IGNORECASE)

In [ ]:
def extract_code(line):
    data = re.search(lineformat, line)
    datadict = data.groupdict()
    return datadict['statuscode'] if 'statuscode' in datadict else None

In [ ]:
extract_code(example_line)

In [ ]:
from collections import Counter
with open('cdn-access.log', 'r') as f:
    lines = (line for line in f)
    codes = (extract_code(line) for line in lines)
    codes_as_list = list(codes)

In [ ]:
cnt = Counter(codes_as_list)

In [ ]:
cnt